# A possible way of finding clusters

Import Acqu and Timepix

In [1]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, Hist2D, histogram, Canvas

Welcome to JupyROOT 6.16/00


Import data and open file

In [2]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Take the data from each detector and get arrays. (does the data always need to be organized beofre using/plotting?)

In [3]:
hitsA = 0
hitsB = 0
TimepixAData = []
TimepixBData = []
TpixEvent=0

def readTimepixData():
    global TpixEvent
    if(aq.epicsEvent==1):           #meaning?
        if(aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')!=TpixEvent): #make sure that we are not reading from the previous buffer
            nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')  #blen for A
            encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
            nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')  #readTimepixBufferFromFile
            encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
            TimepixAData.append(Timepix.Decode(nHitsA,encodedA))
            TimepixBData.append(Timepix.Decode(nHitsB,encodedB))
            TpixEvent = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT') 
        
       
aq.runFunction(readTimepixData,0,50000)
#print(TimepixAData)
#print(TimepixAData(int([['ToA']])))

We want to sort the data by time of arrival. ToA is the third entry  ( so position [2] ). 

In [65]:
def entrySort(pos):
    return pos[2]

Code for pocessing clusters in both detectors. Based on the pseudo code provided by Ken. 

When using buffer, use 'ToA', 'ToT', 'x', 'y' and not [i].

In [99]:
#finds the x and y spread of each cluster.

def spread_position(detector):
    #all for x 
    global spread_x, spread_y
    start_index = cl[0]
    min_x = buf[start_index]["x"]
    max_x = buf[start_index]["x"]
    spread_x = 0.0
    
    n = start_index
    while(n<start_index+cn):            #loop ovber each hit in a cluster to find the lowest time
        if(buf[n]["x"] < min_x):      #if this hit's time is lower than current min value
            min_x = buf[n]["x"]       #set this lower value to min time
        n+=1
          
        
    n = start_index
    while(n<start_index+cn):            #loop ovber each hit in a cluster to find the lowest time
        if(buf[n]["x"] > max_x):      #if this hit's time is lower than current min value
             max_x = buf[n]["x"]       #set this lower value to min time
        n+=1 
    
    
     
    spread_x = max_x - min_x
    #print("Detector A cluster spread: ")
   # print(spread_x, min_x, max_x)
    
    
    #all for y
    start_index = cl[0]
    min_y = buf[start_index]["y"]
    max_y = buf[start_index]["y"]
    spread_y = 0.0 
    
    n = start_index
    while(n<start_index+cn):            #loop ovber each hit in a cluster to find the lowest time
        if(buf[n]["y"] < min_y):      #if this hit's time is lower than current min value
            min_y = buf[n]["y"]       #set this lower value to min time
        n+=1
          
        
    n = start_index
    while(n<start_index+cn):            #loop ovber each hit in a cluster to find the lowest time
        if(buf[n]["y"] > max_y):      #if this hit's time is lower than current min value
             max_y = buf[n]["y"]       #set this lower value to min time
        n+=1 
    
    spread_y = max_y - min_y
    #print("Detector B cluster spread: ")
    #print(spread_y, min_y, max_y)
    
    

In [100]:
#find time spread of each cluster dt

def spread_time(detector):
    global time_spread, max_time, min_time
    start_index = cl[0]              #cl[0] buffer number at that position in array
    min_time = buf[start_index]["ToA"]
    max_time = buf[start_index]["ToA"]

    #to find min_time
    n = start_index
    while(n<start_index+cn):            #loop ovber each hit in a cluster to find the lowest time
        if(buf[n]["ToA"] < min_time):      #if this hit's time is lower than current min value
            min_time = buf[n]["ToA"]       #set this lower value to min time
        n+=1
    #to find max_time
    n = start_index
    while(n<start_index+cn):            #loop ovber each hit in a cluster to find the lowest time
        if(buf[n]["ToA"] > max_time):      #if this hit's time is higher than current min value
            max_time = buf[n]["ToA"]       #set this lower value to max time
        n+=1
        
    time_spread = max_time-min_time
    

In [101]:
def process_cluster(detector):
    start_index = cl[0]              #cl[0] buffer number at that position in array
    min_time = buf[start_index]["ToA"]   #time of arrival of the first hit .ToA or [0]
    dtime = 0.0                          # time difference
    meanx = 0.0
    meany = 0.0
    
    #finding min_time in each cluster and the manx and meany
    n = start_index
    while(n<start_index+cn):            #loop ovber each hit in a cluster to find the lowest time
        if(buf[n]["ToA"] < min_time):      #if this hit's time is lower than current min value
            min_time = buf[n]["ToA"]       #set this lower value to min time
        meanx+=buf[n]["x"]               # add up all the x positions
        meany+=buf[n]['y']               # add up all the y positions
        n+=1                         # move to the next hit in the cluster
    
   
    meanx/=cn        #divide sum of the positions by number of hits to get the mean 
    meany/=cn                        
    
    #plots A
    if(detector=='A'):
        tdposA.Fill(meanx,meany)    #mean x vs mean y plot
        multA.Fill(cn)             #Frequency of hits per cluster
        spreadA.Fill(spread_x,spread_y)   #spread of x vs spread of y
        timespreadA.Fill(time_spread)
        
        #plots
        n = start_index
        while(n<start_index+cn):               # for each hit in the cluster
            dtime = buf[n]["ToA"] - min_time     # find time difference between a hit in the cluster and the min time of the cluster
            dthistA.Fill(dtime)                # put the time difference in the histogram
            dtime_ToTA.Fill(dtime,buf[n]["ToT"])  #2D hist of time diff vs ToT for A data 
            n+=1  
    #plots  B 
    elif(detector=="B"):
        tdposB.Fill(meanx,meany)         # plot mean y vs. mean x
        multB.Fill(cn)                   # plot the counts in the cluster

        #plots
        n = start_index
        while(n<start_index+cn):               #loop through for each hit in the cluster
            dtime = buf[n]["ToA"] - min_time     #time diff from earliest in cluster to each hit
            dthistB.Fill(dtime)                # put the time difference in the histogram
            dtime_ToTB.Fill(dtime,buf[n]["ToT"]) #2D hist of time diff vs ToT for B data 

            n+=1                               #go on to the next hit in that same cluster


Code for processing and graphing clusters. Code for identifying clusters with each detector's data. If hit is within

In [102]:
def cluster_finder(detector):

    global cl,cn,blen,b,nc,buf, spread_x, spread_y
    cl = [None]*100     #hold up to 100 hits in a cluster
    data = []
    cn = 0            #counts number of hits in current cluster
    blen = 0            #length of the current timepix3 buffer
    b = 0               # the running (current?) buffer position
    nc = 0                  # count the number of clusters found
   
    
    if (detector=='A'):
        data = TimepixAData
                               #read each timepix buffer into tp3 (number of entries)
    elif (detector=='B'):
        data = TimepixBData
        
   
    for n in range(len(data)):           #want to go through each buffer in the data
        b=0                              #start at zero
        buf=[]                             #start with a new buffer
        for j in range(len(data[n])):    #each hit in the buffer
            buf.append(data[n][j])       #take hit j and put in current buffer n
        buf.sort(key=entrySort)
        blen = len(buf)
    
    
        while(b<blen):          #for every hit in the buffer (for every buffer position in buffer)
            cl=[None]*100   # starting a new array (so new cluster). Reset counter hits in cluster.
            cn = 0            # zero the cluster counter for the current buffer position
            cl[cn]=b          # assume the current position has the first hit in the cluster and save it
            cn+=1             # increment the counter, since we've counted a hit

            # this next bit was a for loop in Ken's code
            c=b+1             # start at position c that is one above the current buffer position
            while((c<b+99) and (c<blen)):     # check all positions up to 99 away, as long as we're still in the buffer
                if(abs(buf[c]["ToA"]- buf[b]["ToA"]) < 100):   # if an event at c is within 100ns of that at the current buffer position
                    cl[cn] = c                             # save this position to the cluster
                    cn+=1                                  # increment the counter, since we've counted a hit
                else:
                    spread_time(detector)
                    spread_position(detector)
                    process_cluster(detector)               # analyze the cluster we found
                    b+=cn                           # move on beyond this cluster
                    cn = 0                          # reset the counter for hits in the cluster
                    cl = [None]*100                 # reset the array of cluster hits
                    nc+=1                           # increment the number of clusters found

                    if(c<b+99 and c<blen):          # if we're still in the loop
                        cl[cn] = b                  # assume current position is the first in the new cluster
                        cn+=1                       # increment the counter, since we've counted a hit

                c+=1 # move to the next position and go through this loop again
                if(((c>=blen) and cn!=0)or((c>=b+99) and cn!=0)): # if you're exiting the loop but have a cluster to process still
                    process_cluster(detector)
                    b+=cn                  
                    cn = 0                     
                    cl = [None]*100         
                    nc+=1
            b+=1

This plots all four graphs. Calls on cluster_finder() which then calls on the process_cluster().

A data graphs are all on one canvas and B data graphs are on two. 

Below is the code for A data graphs only.

In [112]:
ROOT.enableJSVis()
from ROOT import TF1

# plots for detector A

tdposA = Hist2D(256,0,256,256,0,256)
tdposA.SetTitle("tdposA; Mean x position of cluster; Mean y position of cluster")

multA = Hist(100,0,20)
multA.SetTitle("multA; Number of hits in a cluster; Number of clusters with that amount of hits")

spreadA = Hist2D(256,0,256,256,0,256)
spreadA.SetTitle("spread; spread in x; spread in y")

timespreadA = Hist(100,0,100)
timespreadA.SetTitle("Time spread per cluster dt; time spread dt; clusters")

dthistA = Hist(100,0,100)
dthistA.SetTitle("Time difference;Time difference between a hit and the min time of the cluster; Number of clusters")

dtime_ToTA = Hist2D(100,0,100,250,0,250)
dtime_ToTA.SetTitle("Time diff vs ToT; Time difference between minimum time in a cluster and a hit; ToT")


cA1 = ROOT.TCanvas("myA1","Cluster plots A1", 1000,1000)
cA1.Divide(1,3)
cA2 = ROOT.TCanvas("myA2","Cluster plots A2", 1000,1000)
cA2.Divide(1,3)

cluster_finder('A')


cA1.cd(1)
tdposA.Draw("colz")
cA1.cd(2)
multA.Fit("gaus")
multA.Draw()
cA1.cd(3)
spreadA.Draw("colz")
cA1.Draw()

cA2.cd(1)
timespreadA.Draw()
cA2.cd(2)
dthistA.Draw()
cA2.cd(3)
dtime_ToTA.Draw("colz")
cA2.Draw()

##############

 FCN=2798.09 FROM MIGRAD    STATUS=CONVERGED     123 CALLS         124 TOTAL
                     EDM=2.73211e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.56345e+04   1.37872e+02   2.35733e+00  -6.42226e-06
   2  Mean         1.63758e+00   8.50772e-03   1.44999e-04  -1.60611e-02
   3  Sigma        8.21423e-01   8.90195e-03   3.37368e-05  -1.15463e-01


In [48]:
#Making the fourth graph 2D and making a Gaussian fit using FitSlicesY

#dtime_ToTA

#Fit slices projected along Y fron bins in X [0,7] with more than 20 bins in Y filled

outfits = ROOT.TObjArray()
cA3 = ROOT.TCanvas("myA3","Least squares of gaussian 2D histogram A3", 800,800)
dtime_ToTA.FitSlicesY(0,0,4,4,"R",outfits)
dtime_ToTA.SetTitle("Time diff vs mean ToT; Time difference; mean ToT")
print dtime_ToTA.GetName()
outfits[1].Fit("gaus")
outfits[1].Draw()

cA3.Draw()



Hist2D_3AXCxZVSrb4ZTqFx3Zxdzi
 FCN=2627.15 FROM MIGRAD    STATUS=CONVERGED      67 CALLS          68 TOTAL
                     EDM=2.43809e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.99912e+02   2.47770e+00   5.16689e-02  -1.10914e-05
   2  Mean         7.93057e+01   2.29075e-01   5.22812e-03  -6.14486e-04
   3  Sigma        3.82412e+01   1.51374e-01   2.57609e-05   7.99923e-02
 FCN=1215.76 FROM MIGRAD    STATUS=CONVERGED      69 CALLS          70 TOTAL
                     EDM=4.8288e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.25202e+02   2.86477e+00   3.54554e-02   8.44246e-05
   2  Mean         2.72724e+01   3.06604e-01   4.45236e-03  -1.98796e-04
   3

In [9]:
#Calculate the mean frequency of each bin by hand and plot these results with a Gaussian fit.

cA4 = ROOT.TCanvas("cA4","My cA4", 800,800)
gA = ROOT.TGraph()
mean = [0]*4
entries = [0]*4
x=[]
y=[]

for bn in range (1,5):
    for i in range(256):
        entries[bn-1]+=dtime_ToTA.GetBinContent(bn,i)
        mean[bn-1]+= (dtime_ToTA.GetBinContent(bn,i))*(i-1)
    mean[bn-1]/=entries[bn-1]
    x.append(bn)
    y.append(mean[bn-1])
    gA.SetPoint(bn-1,x[bn-1],y[bn-1])
    
gA.GetXaxis().SetRange(0,100)
gA.GetYaxis().SetRange(0,256)
gA.Fit("gaus")
gA.Draw("C")

#f1 = ROOT.TF1("f1", "(amplitude*exp(-0.5*((x-Mean)/Sigma)*((x-Mean)/Sigma)))",0,20)

cA4.Draw()

print(mean)

[79.4575744003185, 32.75524421394845, 18.534562211981566, 9.22082191780822]

****************************************
         Invalid FitResult  (status = 4 )
****************************************
Minimizer is Minuit / Migrad
Chi2                      =      24.5061
NDf                       =            1
Edm                       =    0.0333037
NCalls                    =         1351
Constant                  =       116779   +/-   133388      
Mean                      =     -18.5244   +/-   4.77991     
Sigma                     =       5.1077   +/-   0.898606     	 (limited)


In [10]:
%%cpp 
 void fitslicesy() {


//TString dir = gROOT->GetTutorialDir();
//dir.Append("/hsimple.C");
//dir.ReplaceAll("/./","/");
//if (!gInterpreter->IsLoaded(dir.Data())) gInterpreter->LoadMacro(dir.Data());
//TFile *hsimpleFile = (TFile*)gROOT->ProcessLineFast("hsimple(1)");
//if (!hsimpleFile) return;
//TH2F *hpxpy = (TH2F*)hsimpleFile->Get("hpxpy");


dtime_ToTA = new TH2D(100,0,100,250,0,250)
dtime_ToTA.Fill(dtime,dataA[n][1])


TCanvas *c1 = new TCanvas("c1","c1",700,500);
c1->SetFillColor(42);
c1->Divide(2,1);
TPad *leftPad = (TPad*)c1->cd(1);;
leftPad->Divide(1,2);


leftPad->cd(1);
gPad->SetTopMargin(0.12);
gPad->SetFillColor(33);
dtime_ToTA->Draw();
dtime_ToTA->GetXaxis()->SetLabelSize(0.06);
dtime_ToTA->GetYaxis()->SetLabelSize(0.06);
dtime_ToTA->SetMarkerColor(kYellow);


dtime_ToTA->FitSlicesY(0,7,32,20);


leftPad->cd(2);
gPad->SetFillColor(33);
TH2F *dtime_ToTA_0 = (TH2F*)hsimpleFile->Get("dtime_ToTA_0");
dtime_ToTA_0->Draw();
TPad *rightPad = (TPad*)c1->cd(2);
rightPad->Divide(1,2);
rightPad->cd(1);
gPad->SetTopMargin(0.12);
gPad->SetLeftMargin(0.15);
gPad->SetFillColor(33);
TH2F *dtime_ToTA_1 = (TH2F*)hsimpleFile->Get("dtime_ToTA_1");
hpxpy_1->Draw();


rightPad->cd(2);
gPad->SetTopMargin(0.12);
gPad->SetLeftMargin(0.15);
gPad->SetFillColor(33);
TH2F *hpxpy_2 = (TH2F*)hsimpleFile->Get("dtime_ToTA_2");
dtime_ToTA_2->SetMinimum(0.8);
dtime_ToTA_2->Draw();

}

input_line_174:13:1: error: use of undeclared identifier 'dtime_ToTA'
dtime_ToTA = new TH2D(100,0,100,250,0,250)
^
input_line_174:13:18: error: no matching constructor for initialization of 'TH2D'
dtime_ToTA = new TH2D(100,0,100,250,0,250)
                 ^    ~~~~~~~~~~~~~~~~~~~
/cern/root/v6.16.00/build/include/TH2.h:301:4: note: candidate constructor not viable: no known conversion from 'int' to 'const char *' for 1st argument
   TH2D(const char *name,const char *title,Int_t nbinsx,const Double_t *xbins
   ^
/cern/root/v6.16.00/build/include/TH2.h:303:4: note: candidate constructor not viable: no known conversion from 'int' to 'const char *' for 1st argument
   TH2D(const char *name,const char *title,Int_t nbinsx,const Float_t  *xbins
   ^
/cern/root/v6.16.00/build/include/TH2.h:297:4: note: candidate constructor not viable: requires 7 arguments, but 6 were provided
   TH2D(const char *name,const char *title,Int_t nbinsx,const Double_t *xbins
   ^
/cern/root/v6.16.00/build/include/

In [11]:
%%cpp
 void fitslicesy()
{
dtime_ToTA->FitSlicesY(0,15,22,10);
dtime_ToTA->Draw(); 
}

input_line_175:3:1: error: use of undeclared identifier 'dtime_ToTA'
dtime_ToTA->FitSlicesY(0,15,22,10);
^
input_line_175:4:1: error: use of undeclared identifier 'dtime_ToTA'
dtime_ToTA->Draw(); 
^


In [12]:

#dtime_ToTA

#Fit slices projected along Y fron bins in X [0,7] with more than 20 bins in Y filled
#dtime_ToTA.FitSlicesY(0,0,7,20)
outfits = ROOT.TObjArray()
cA3 = ROOT.TCanvas("myA3","Least squares of gaussian 2D histogram A3", 800,800)
dtime_ToTA.FitSlicesY(0,0,10,20,"R",outfits)
dtime_ToTA.SetTitle("Time diff vs mean ToT; Time difference; mean ToT")
print dtime_ToTA.GetName()
dtime_ToTA.Fit("gaus")
outfits[1].Draw()

cA3.Draw()

Hist2D_TXkHK8hMBzP7u8yMK6WVBW


 FCN=2627.15 FROM MIGRAD    STATUS=CONVERGED      71 CALLS          72 TOTAL
                     EDM=2.43813e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.99912e+02   2.47770e+00   5.16689e-02  -1.10916e-05
   2  Mean         7.93057e+01   2.29075e-01   5.22812e-03  -6.14487e-04
   3  Sigma        3.82412e+01   1.51374e-01   2.57609e-05   7.99934e-02
 FCN=1215.76 FROM MIGRAD    STATUS=CONVERGED      69 CALLS          70 TOTAL
                     EDM=4.82879e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.25202e+02   2.86477e+00   3.54554e-02   8.44244e-05
   2  Mean         2.72724e+01   3.06604e-01   4.45236e-03  -1.98796e-04
   3  Sigma        2.40254e+01   

In [13]:
#Create a canvas and divide it
c1=ROOT.TCanvas("myA3","Least squares of gaussian 2D histogram A3",700,500)

c1.SetFillColor(42)
c1.Divide(2,1)
c1.cd(1)
leftPad = ROOT.TPad("leftPad","",0,0.94,1,1)
leftPad.Divide(1,2)

dtime_ToTA = Hist2D(100,0,100,250,0,250)
dtime_ToTA.SetTitle("Time diff vs ToT; Time difference between minimum time in a cluster and a hit; ToT")

#draw original 2D histogram
leftPad.cd(1)
gPad.SetTopMargin(0.12)
gpad.SetFillColor(33)
dtime_ToTA.Draw()
dtime_ToTA.GetXaxis().SetLabelSize(0.06)
dtime_ToTA.GetYaxis().SetLabelSize(0.06)
dtime_ToTA.SetMarkerColor(kYellow)

#Fit slices projected along Y fron bins in X [7,32]
dtime_ToTA.FitSlicesY(0,0,0,20)

#Show fitted "mean" for each slice
leftPad.cd(2)
gPad.SetFillColor(33)
dtime_ToTA_0.Draw()
c1.cd(2)
rightPad = ROOT.TPad.GPad("rightPad","",0,0.94,1,1)
rightPad.Divide(1,2)
rightPad.cd(1)
gPad.SetTopMargin(0.12)
gPad.SetLeftMargin(0.15)
gPad.SetFillColor(33)
dtime_ToTA_1.Draw()


#Show fitted "sigma" for each slice
rightPad.cd(2)
gPad.SetTopMargin(0.12)
gPad.SetLeftMargin(0.15)
gPad.SetFillColor(33)
dtime_ToTA_2.SetMinimum(0.8)
dtime_ToTA_2.Draw(

SyntaxError: invalid syntax (<ipython-input-13-4faece5eebde>, line 45)

In [ ]:
void FitSlicesY(TF1 *f1, Int_t binmin, Int_t binmax, Int_t cut, Option_t *option) 


In [ ]:
#plot for B data
##############

tdposB = Hist2D(256,0,256,256,0,256)
tdposB.SetTitle("tdposB; Mean x position of cluster; Mean y position of cluster")


multB = Hist(50,0,50)
multB.SetTitle("multB; Number of hits in a cluster; Number of clusters with taht amount of hits")


dthistB = Hist(100,0,100)
dthistB.SetTitle("Time difference;Time difference between a hit and the min time of the cluster; Number of clusters")


dtime_ToTB = Hist2D(100,0,100,250,0,250)
dtime_ToTB.SetTitle("Time diff vs ToT; Time difference between minimum time in a cluster and a hit; ToT")


cB1 = ROOT.TCanvas("myB1","Cluster plots B1", 1000,1000)
cB1.Divide(1,2)
cB2 = ROOT.TCanvas("myB2","Cluster plots B2", 1000,1000)
cB2.Divide(1,2)

cluster_finder('B')

cB1.cd(1)
tdposB.Draw("colz")
cB1.cd(2)
multB.Fit("gaus")
multB.Draw()
####################
cB2.cd(1)
dthistB.Fit("gaus")
dthistB.Draw()
cB2.cd(2)
#dtime_ToTB.Fit("gaus")
dtime_ToTB.Draw("colz")
cB1.Draw()
cB2.Draw()

# Graphing equations in jupyter notebook.